In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

test_frames = []
for i in range(20):
    frame = cv2.imread('test_frames\\frame_' + str(i) + '.png') 
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    test_frames.append(frame)
print(len(test_frames))

In [ ]:
# frame:      input frame from CMOS camera
# resolution: CMOS image resolution
# beta:       threshold on potential hit detection
# gamma:      threshold on averaged Frame pixels value sum
# delta:      threshold on fraction of black pixels count

def alg1(frames: list[np.array], resolution: tuple[int ,int], beta, gamma, delta=0.04):
    potential_hits = []
    (Y, X) = resolution
    for frame in frames:
        r,g,b = cv2.split(frame.astype(float))
        frame_sum = r + g + b
        max_frame_sum = np.max(frame_sum)
        zero_count_frame_sum = 0
        for j in range(Y):
            for i in range(X):
                zero_count_frame_sum += 1 if frame_sum[j, i] == 0 else 0
        sum_frame_sum = frame_sum.sum()
        avg_frame_sum = sum_frame_sum / (Y*X)
        blacks_frame_sum = zero_count_frame_sum / (Y*X)
        if max_frame_sum > beta and avg_frame_sum < gamma and blacks_frame_sum > delta:
            potential_hits.append(frame)
    return potential_hits




In [ ]:
# resolution: CMOS image resolution
# zone_size:  size of square block on which input frame will be divided

def alg2(frames: list[np.array], resolution: tuple[int ,int], zone_size):
    potential_hits = []
    trigger_map_width = resolution[1] // zone_size
    trigger_map_height = resolution[0] // zone_size
    heat = np.zeros(resolution)
    pixel_scores = np.zeros(resolution)
    threshold = 0
    for frame in frames:
        r,g,b = cv2.split(frame.astype(float))
        total = r + g + b

        diff = total - heat

        for b in range(resolution[0]):
            for a in range(resolution[1]):
                if total[b, a] > 4 and diff[b, a] > 4 * heat[b, a]:
                    pixel_scores[b, a] = diff[b, a] * diff[b, a]
                else:
                    pixel_scores[b, a] = 0
                heat[b, a] = 0.03 * total[b, a] + (1 - 0.03) * heat[b, a]

        block_scores = np.zeros((trigger_map_height, trigger_map_width))
        for b in range(resolution[0]):
            for a in range(resolution[1]):
                block_scores[b // zone_size, a // zone_size] += pixel_scores[b, a]

        un = np.sort(block_scores, axis=None)
        max2_nd = un[len(un) - 2]
        
        if threshold == 0:
            un = np.sort(block_scores, axis=None)
            threshold = max2_nd
        else:
            threshold = 0.1 * max2_nd + (1 - 0.1) * threshold
        
        flag = False
        for b in range(1, trigger_map_height - 1):
            for a in range(1, trigger_map_width - 1):
                if block_scores[b, a] > 2 * threshold:
                    potential_hits.append(frame)
                    flag = True
                    break
            if flag:
                break
    return potential_hits

In [ ]:
def max_pooling(frame, num):
    rows = frame.shape[0] // num
    cols = frame.shape[1] // num
    frame_pool = np.zeros((rows, cols))
    
    for j in range(rows):
        for i in range(cols):
            frame_pool[j, i] = np.max(frame[j * num: j * num + num, i * num: i * num + num])
    return frame_pool

# edge:       width of image edge in pixel which will be cropped during processing
# kernel:     gaussian kernel matrix
# imc:        number of initial iterations devoted to initialise moving average
# resolution: CMOS image resolution
# alfa:       moving average memory
# theta:      threshold of potential hit detections

def alg3(frames: list[np.array], resolution: tuple[int ,int], edge, kernel, imc, alfa, theta):
    potential_hits = []
    frame_avg = None
    for frame in frames:
        r, g, b = cv2.split(frame.astype(float))
        frame_sum = r + g + b
        frame_crop = frame_sum[edge:resolution[0] - edge, edge:resolution[1] - edge] 
        frame_pool = max_pooling(frame_crop, 2)
        frame_gaussian = cv2.GaussianBlur(frame_pool, kernel, 0)
        
        flag = False
        if imc <= 0:
            for b in range(frame_gaussian.shape[0]):
                for a in range(frame_gaussian.shape[1]):
                    if frame_avg[b, a] * 4 < frame_gaussian[b, a] and frame_pool[b, a] > theta:
                        potential_hits.append(frame)
                        flag = True
                        break
                if flag:
                    break
        else:
            imc -= 1

        if frame_avg is None:
            frame_avg = frame_gaussian
        else:
            frame_avg = (alfa - 1) * frame_avg + alfa * frame_gaussian
    return potential_hits

In [ ]:
hits_alg1 = alg1(test_frames, (1080, 1920), 120, 40)
hits_alg2 = alg2(test_frames, (1080, 1920), 20)
hits_alg3 = alg3(test_frames, (1080, 1920), 30, (3, 3), 12, 0.2, 255)

In [ ]:
print(f"Algorithm 1. detected {len(hits_alg1)}. possible hits")
print(f"Algorithm 2. detected {len(hits_alg2)}. possible hits")
print(f"Algorithm 3. detected {len(hits_alg3)}. possible hits")